In [4]:
pip install pillow

In [9]:
import os
import random
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


DATA_DIR = r"E:\Download\CVPRStudentDataset" 
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 20
SEED = 42
VAL_SPLIT = 0.2

# only these formats allowed
ALLOWED_EXT = (".jpg", ".jpeg", ".png", ".bmp", ".webp", ".gif")

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


def is_valid_image(path: str) -> bool:
    """Return True if file is a valid readable image."""
    try:
        with Image.open(path) as im:
            im.verify()  # quick corruption check
        return True
    except Exception:
        return False

# class folders
class_names = sorted([
    d for d in os.listdir(DATA_DIR)
    if os.path.isdir(os.path.join(DATA_DIR, d))
])

if len(class_names) == 0:
    raise RuntimeError("No class folders found. Check DATA_DIR path.")

class_to_idx = {c: i for i, c in enumerate(class_names)}

all_paths = []
all_labels = []
bad_files = []

for cls in class_names:
    folder = os.path.join(DATA_DIR, cls)
    for fn in os.listdir(folder):
        path = os.path.join(folder, fn)

        # skip non-files
        if not os.path.isfile(path):
            continue

        # skip unsupported extensions (thumbs.db, txt, etc.)
        if not fn.lower().endswith(ALLOWED_EXT):
            continue

        # verify image
        if is_valid_image(path):
            all_paths.append(path)
            all_labels.append(class_to_idx[cls])
        else:
            bad_files.append(path)

print(f"Classes: {len(class_names)}")
print(f"Valid images: {len(all_paths)}")
print(f"Skipped bad/corrupt images: {len(bad_files)}")

# Save bad list (so you can delete later if you want)
if bad_files:
    with open("bad_files.txt", "w", encoding="utf-8") as f:
        for p in bad_files:
            f.write(p + "\n")
    print("Saved bad file list -> bad_files.txt")

if len(all_paths) == 0:
    raise RuntimeError("No valid images found after filtering. Check dataset files.")



idx = np.arange(len(all_paths))
np.random.shuffle(idx)

val_size = int(VAL_SPLIT * len(all_paths))
val_idx = idx[:val_size]
train_idx = idx[val_size:]

train_paths = [all_paths[i] for i in train_idx]
train_labels = [all_labels[i] for i in train_idx]
val_paths = [all_paths[i] for i in val_idx]
val_labels = [all_labels[i] for i in val_idx]

print(f"Train: {len(train_paths)} | Val: {len(val_paths)}")


def load_and_preprocess(path, label):
    img_bytes = tf.io.read_file(path)
    img = tf.io.decode_image(img_bytes, channels=3, expand_animations=False)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.shuffle(2000, seed=SEED, reshuffle_each_iteration=True)
train_ds = train_ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds = val_ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)


data_aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.10),
])

model = keras.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    data_aug,

    layers.Conv2D(32, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(len(class_names), activation="softmax"),
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


callbacks = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint("cnn_face_model.h5", save_best_only=True),
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

with open("class_names.txt", "w", encoding="utf-8") as f:
    for c in class_names:
        f.write(c + "\n")

print("Saved: cnn_face_model.h5 and class_names.txt")


Classes: 40
Valid images: 679
Skipped bad/corrupt images: 0
Train: 544 | Val: 135


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)       │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 40)             │        10,280 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,492,392 (32.40 MB)

 Trainable params: 8,492,392 (32.40 MB)

...

17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.8033 - loss: 0.6774 - val_accuracy: 0.8000 - val_loss: 0.6033
Epoch 10/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 903ms/step - accuracy: 0.8081 - loss: 0.5457 

17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.7941 - loss: 0.5868 - val_accuracy: 0.8519 - val_loss: 0.4972
Epoch 11/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.8566 - loss: 0.4967 - val_accuracy: 0.7481 - val_loss: 0.6015
Epoch 12/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 904ms/step - accuracy: 0.8454 - loss: 0.5099 

17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.8529 - loss: 0.5076 - val_accuracy: 0.9037 - val_loss: 0.4677
Epoch 13/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.8621 - loss: 0.4288 - val_accuracy: 0.9037 - val_loss: 0.4992
Epoch 14/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.8824 - loss: 0.3760 - val_accuracy: 0.8963 - val_loss: 0.4853
Epoch 15/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 900ms/step - accuracy: 0.8972 - loss: 0.3277 

17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9007 - loss: 0.3219 - val_accuracy: 0.8963 - val_loss: 0.4462
Epoch 16/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 907ms/step - accuracy: 0.9004 - loss: 0.3291 

17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9099 - loss: 0.2894 - val_accuracy: 0.9037 - val_loss: 0.3433
Epoch 17/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9265 - loss: 0.2378 - val_accuracy: 0.9185 - val_loss: 0.4459
Epoch 18/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9320 - loss: 0.2401 - val_accuracy: 0.9259 - val_loss: 0.3470
Epoch 19/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9467 - loss: 0.2003 - val_accuracy: 0.9111 - val_loss: 0.4599
Epoch 20/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9338 - loss: 0.1969 - val_accuracy: 0.8889 - val_loss: 0.6690
Saved: cnn_face_model.h5 and class_names.txt


In [6]:
import os, cv2

DATA_DIR = r"E:\Download\CVPRStudentDataset"  
NUM = 20

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

student_id = input("Enter new student ID (e.g., 22-99999-1): ").strip()
save_dir = os.path.join(DATA_DIR, student_id)
os.makedirs(save_dir, exist_ok=True)

cap = cv2.VideoCapture(0)
count = 0

print("SPACE = capture, Q = quit")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    cv2.putText(frame, f"{student_id} {count}/{NUM}", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
    cv2.imshow("Register", frame)

    k = cv2.waitKey(1) & 0xFF
    if k in [ord("q"), ord("Q")]:
        break

    if k == 32:  # SPACE
        if len(faces) == 0:
            print("No face detected. Try again.")
            continue
        (x,y,w,h) = max(faces, key=lambda b: b[2]*b[3])
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (256,256))

        out = os.path.join(save_dir, f"{count+1:02d}.jpg")
        cv2.imwrite(out, face)
        count += 1
        print("Saved:", out)

        if count >= NUM:
            print("Registration complete.")
            break

cap.release()
cv2.destroyAllWindows()

print("\nIMPORTANT: New student added -> retrain model:")
print("python 01_train_cnn.py")





Enter new student ID (e.g., 22-99999-1):  22-49538-3


SPACE = capture, Q = quit
Saved: E:\Download\CVPRStudentDataset\22-49538-3\01.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\02.jpg
No face detected. Try again.
Saved: E:\Download\CVPRStudentDataset\22-49538-3\03.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\04.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\05.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\06.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\07.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\08.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\09.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\10.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\11.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\12.jpg
No face detected. Try again.
No face detected. Try again.
Saved: E:\Download\CVPRStudentDataset\22-49538-3\13.jpg
No face detected. Try again.
Saved: E:\Download\CVPRStudentDataset\22-49538-3\14.jpg
Saved: E:\Download\CVPRStudentDataset\22-49538-3\15.jpg
No face detected. 

In [13]:
import cv2, numpy as np, pandas as pd
from datetime import datetime
from tensorflow import keras

MODEL_PATH = "cnn_face_model.h5"
CLASSES_TXT = "class_names.txt"
ATT_CSV = "attendance.csv"
CONF = 0.70
IMG_SIZE = (128, 128)

model = keras.models.load_model(MODEL_PATH)

with open(CLASSES_TXT, "r", encoding="utf-8") as f:
    class_names = [x.strip() for x in f if x.strip()]

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def preprocess(face_bgr):
    face = cv2.resize(face_bgr, IMG_SIZE)
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = face.astype("float32") / 255.0
    return np.expand_dims(face, axis=0)

def mark(student_id):
    now = datetime.now()
    row = {"student_id": student_id,
           "date": now.strftime("%Y-%m-%d"),
           "time": now.strftime("%H:%M:%S")}

    try:
        df = pd.read_csv(ATT_CSV)
    except FileNotFoundError:
        df = pd.DataFrame(columns=["student_id","date","time"])

    # one per day
    if ((df["student_id"] == student_id) & (df["date"] == row["date"])).any():
        return False

    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(ATT_CSV, index=False)
    return True

cap = cv2.VideoCapture(0)
last_name, last_conf = "No face", 0.0

print("M = mark, Q = quit")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    if len(faces) > 0:
        (x,y,w,h) = max(faces, key=lambda b: b[2]*b[3])
        face = frame[y:y+h, x:x+w]

        probs = model.predict(preprocess(face), verbose=0)[0]
        idx = int(np.argmax(probs))
        conf = float(probs[idx])

        name = class_names[idx] if conf >= CONF else "Unknown"
        last_name, last_conf = name, conf

        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    cv2.putText(frame, f"{last_name} ({last_conf:.2f})", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
    cv2.imshow("Attendance", frame)

    k = cv2.waitKey(1) & 0xFF
    if k in [ord("q"), ord("Q")]:
        break
    if k in [ord("m"), ord("M")]:
        if last_name not in ["Unknown", "No face"]:
            ok = mark(last_name)
            print("Marked!" if ok else "Already marked today.")
        else:
            print("Unknown / No face -> not marked.")

cap.release()
cv2.destroyAllWindows()
print("Saved report:", ATT_CSV)


M = mark, Q = quit
Already marked today.
Already marked today.
Already marked today.
Already marked today.
Already marked today.
Saved report: attendance.csv


In [ ]:
import pandas as pd

ATT_CSV = "attendance.csv"

try:
    df = pd.read_csv(ATT_CSV)
except FileNotFoundError:
    print("attendance.csv not found. Run 03_mark_attendance.py first.")
    raise SystemExit

print("Commands: summary | all | <student_id> | exit")

while True:
    q = input("\nEnter: ").strip()
    if q.lower() in ["exit","quit","q"]:
        break
    if q.lower() == "all":
        print(df.sort_values(["date","time"], ascending=False).to_string(index=False))
        continue
    if q.lower() == "summary":
        s = df.groupby("student_id")["date"].nunique().reset_index(name="days_present")
        print(s.sort_values("days_present", ascending=False).to_string(index=False))
        continue

    sub = df[df["student_id"] == q].sort_values(["date","time"], ascending=False)
    if sub.empty:
        print("No record found.")
    else:
        print(sub[["date","time"]].to_string(index=False))


Commands: summary | all | <student_id> | exit



Enter:  22-48000-3


      date     time
2026-01-06 02:00:56



Enter:  all


student_id       date     time
22-48000-3 2026-01-06 02:00:56
